# Browser-Use SDK를 활용한 기본 브라우저 도구 사용

## 개요

이 튜토리얼에서는 오픈 소스 Browser-Use SDK를 Amazon Bedrock Agentcore 브라우저 도구와 함께 사용하는 방법을 알아봅니다. 브라우저 도구를 헤드리스로 사용하고 브라우저를 실시간으로 확인하는 예제를 제공합니다.

### 튜토리얼 세부 정보

| 정보 | 세부 정보 |
|:--------------------|:-------------------------------------------------------------------------------------------
| 튜토리얼 유형 | 대화형 |
| 에이전트 유형 | 단일 |
| Agentic Framework | Broeser-Use |
| LLM 모델 | Anthropic Claude 3.7 Sonnet |
| 튜토리얼 구성 요소 | Browser-Use SDK를 사용하여 헤드리스 방식으로 Bedrock Agentcore 브라우저 도구와 상호 작용 |
| 튜토리얼 수직 | 교차 수직 |
| 예제 복잡성 | 쉬움 |
| 사용된 SDK | Amazon BedrockAgentCore Python SDK, Browser-Use |

### 튜토리얼 아키텍처

이 튜토리얼에서는 Agentcore 브라우저 도구와 함께 Browser-Use SDK를 사용하는 방법을 설명합니다.

이 예시에서는 Browser-Use 에이전트에 자연어 명령을 전송하여 Bedrock Agentcore 브라우저에서 헤드리스 방식으로 작업을 수행합니다.

<div style="text-align:left">
<img src="images/browser-tool.png" width="50%"/>
</div>

### 튜토리얼 주요 기능

* 헤드리스 방식으로 브라우저 도구 사용
* 브라우저 도구와 함께 Browser-Use 사용

## Prerequisites

To execute this tutorial you will need:
* Python 3.11+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Browser-Use SDK 

## 작동 방식

브라우저 도구 샌드박스는 AI 에이전트가 웹 브라우저와 안전하게 상호 작용할 수 있도록 하는 안전한 실행 환경입니다. 사용자가 요청하면 LLM(Large Language Model)이 적절한 도구를 선택하고 명령을 번역합니다. 이러한 명령은 헤드리스 브라우저와 Playwright와 같은 도구를 사용하는 호스팅 라이브러리 서버가 포함된 제어된 샌드박스 환경에서 실행됩니다. 샌드박스는 제한된 공간 내에서 웹 상호 작용을 제한하여 무단 시스템 접근을 방지함으로써 격리 및 보안을 제공합니다. 에이전트는 스크린샷을 통해 피드백을 받고 시스템 보안을 유지하면서 자동화된 작업을 수행할 수 있습니다.

![architecture local](../images//browser-tool.png)

## 1. 환경 설정
먼저, 브라우저 도구 샌드박스 클라이언트를 초기화하는 데 필요한 라이브러리를 설치하고 임포트해 보겠습니다.

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
from bedrock_agentcore.tools.browser_client import BrowserClient
from browser_use import Agent
from browser_use.browser.session import BrowserSession
from browser_use.browser import BrowserProfile
from langchain_aws import ChatBedrockConverse
from rich.console import Console
from contextlib import suppress
import asyncio

In [ ]:
console = Console()

## 2. 브라우저 클라이언트 설정
브라우저 클라이언트를 설정하고 준비가 될 때까지 기다립니다. 그런 다음 웹 소켓 URL과 헤더를 생성합니다.

In [ ]:
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

client = BrowserClient(region)
client.start()

# Extract ws_url and headers
ws_url, headers = client.generate_ws_headers()

## 4. 브라우저 작업 실행을 위한 도우미 함수
browser-use Agent를 사용하여 브라우저 자동화 작업을 실행합니다.

In [ ]:
async def run_browser_task(browser_session: BrowserSession, bedrock_chat: ChatBedrockConverse, task: str) -> None:
    """
    Run a browser automation task using browser_use
    
    Args:
        browser_session: Existing browser session to reuse
        bedrock_chat: Bedrock chat model instance
        task: Natural language task for the agent
    """
    try:
        # Show task execution
        console.print(f"\n[bold blue]🤖 Executing task:[/bold blue] {task}")
        
        # Create and run the agent
        agent = Agent(
            task=task,
            llm=bedrock_chat,
            browser_session=browser_session
        )
        
        # Run with progress indicator
        with console.status("[bold green]Running browser automation...[/bold green]", spinner="dots"):
            await agent.run()
        
        console.print("[bold green]✅ Task completed successfully![/bold green]")
        
    except Exception as e:
        console.print(f"[bold red]❌ Error during task execution:[/bold red] {str(e)}")
        import traceback
        if console.is_terminal:
            traceback.print_exc()

## 5. 브라우저 사용 프로필을 사용하여 브라우저 작업 호출
CDP WebSocket 연결을 사용하여 영구 브라우저 세션을 생성하고 자동화된 웹 작업을 위한 Bedrock Claude 모델을 초기화합니다. 적절한 정리를 통해 세션 수명 주기를 관리하고 AI 기반 명령을 통해 브라우저 자동화 작업을 실행합니다.

In [ ]:
# Create persistent browser session and model
browser_session = None
bedrock_chat = None

try:
    # Create browser profile with headers
    browser_profile = BrowserProfile(
        headers=headers,
        timeout=1500000,  # 150 seconds timeout
    )
    
    # Create a browser session with CDP URL and keep_alive=True for persistence
    browser_session = BrowserSession(
        cdp_url=ws_url,
        browser_profile=browser_profile,
        keep_alive=True  # Keep browser alive between tasks
    )
    
    # Initialize the browser session
    console.print("[cyan]🔄 Initializing browser session...[/cyan]")
    await browser_session.start()
    
    # Create ChatBedrockConverse once
    bedrock_chat = ChatBedrockConverse(
        model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        region_name="us-west-2"
    )
    
    console.print("[green]✅ Browser session initialized and ready for tasks[/green]\n")

    task = "Amazon.com에서 Macbook을 검색하고 첫 번째 Macbook의 세부 정보를 추출합니다."

    await run_browser_task(browser_session, bedrock_chat, task)

finally:
    # Close the browser session
    if browser_session:
        console.print("\n[yellow]🔌 Closing browser session...[/yellow]")
        with suppress(Exception):
            await browser_session.close()
        console.print("[green]✅ Browser session closed[/green]")


## 6. 정리
브라우저 세션이 아직 종료되지 않았다면 종료하세요.

In [ ]:
client.stop()
print("Browser session stopped successfully!")